In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import http

drive.mount('/content/drive')

# 시가총액 상위 300개 기업리스트 (미래에셋 대우 API 미확인으로, 2021.09.12 기준 시총 상위 300개 기업 사용)
lists = pd.read_csv("/content/drive/MyDrive/미래에셋/kospi/KOSPI_시가총액_상위500.csv")
lists_1 = lists.loc[0:299]
lists_2 = lists_1[['종목명']]


# ESG 키워드
ESG = {
'E_keyword' : ['친환경', '탄소', '에너지'],
'S_keyword' : ['캠페인', '교육', '사회'],
'G_keyword' : ['위원회', '이사', '지배구조']}
ESG_keyword = pd.DataFrame.from_dict(ESG)

print(ESG_keyword)


Mounted at /content/drive
  E_keyword S_keyword G_keyword
0       친환경       캠페인       위원회
1        탄소        교육        이사
2       에너지        사회      지배구조


In [ ]:
# 지난 N개월 간의 종목명 + E 관련 뉴스기사 본문 추출
from multiprocessing import Pool
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
import urllib.request
from urllib.parse import quote
import pandas as pd


# 크롤링 메인 함수
def get_news(stock, keyword, query, sort, lastpage, period):

  stockName = stock
  ESGKeyword = keyword

  # 크롤링 결과 저장 데이터프레임
  news_df = pd.DataFrame(columns=("stockName", "ESGKeyword", "Title", "Article")) 

  idx = 0


  for month in range (0, period): # 해당 검색어(query)로 N(=period)개월치 기사 추출

    s_date = datetime.date.today() + timedelta(days = -30 * (month+1))
    e_date = datetime.date.today() + timedelta(days = -30 * month)

    pageNum = 1

    for i in range (1, int(lastpage)*10, 10):
      # print(f"/{pageNum}페이지입니다/")

      url_query = quote(query)
      url = f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={url_query}&sort=0&photo=0&field=0&pd=6&ds={s_date}&de={e_date}&nso=so:r,p:{period}m,a:all&start={i}"

      search_url = urllib.request.urlopen(url).read()
      soup = BeautifulSoup(search_url, 'html.parser')
      links = soup.find_all('div', {'class':'info_group'})
  

      # 하나의 검색 페이지 안에서의 반복
      for link in links:

        # '네이버 뉴스' 형식으로 작성된 기사만을 추출 (href 2번)
        news_urls = link.find_all('a')

        if (len(news_urls) == 1): # '네이버 뉴스' 형식 X
          continue

        else:  # '네이버 뉴스' 형식 O
          
          for news_url_test in news_urls:
            news_url = news_url_test.get('href')

          try:
            news_link = urllib.request.urlopen(news_url).read()
          except (http.client.IncompleteRead) as e:
            news_link = e.partial
          
          news_html = BeautifulSoup(news_link, 'html.parser')

          try: 
            title = news_html.find('h3', {'id': 'articleTitle'}).get_text()
            article = news_html.find('div', {'id': 'articleBodyContents'}).get_text()
            article = article.replace("// flash 오류를 우회하기 위한 함수 추가", "")
            article = article.replace("function _flash_removeCallback() {}", "")
            article = article.replace("\n","")
            article = article.replace("\t","")

            news_df.loc[idx] = [stockName, ESGKeyword, title, article]
            idx += 1

          except: 
            continue

          # print("#", end="")


      pageNum += 1


  return news_df




In [ ]:
# E(환경)부문


# CONCAT 목적의 초기 데이터프레임 형성
default = {
'stockName' : ['Test'],
'ESGKeyword' : ['Test'],
'Title' : ['Test'],
'Article' : ['Test']}
df_default = pd.DataFrame.from_dict(default) 


# 파라미터 입력 후 크롤링
for i in range(25,50): # 300: 시총 상위 50개 기업

  query = lists_2['종목명'].values[i] + "+" + ESG_keyword['E_keyword'].values[1]
  sort = "0" # 관련도 순 기사
  lastpage = 100 # 100 페이지까지 크롤링 (최대 400페이지까지 가능하기는 하나, 파일 크기 과부하 방지)
  period = 6 # 6개월간의 기사

  print(lists_2['종목명'].values[i])
  print(ESG_keyword['E_keyword'].values[1])
  print(i)
  
  df = get_news(lists_2['종목명'].values[i], ESG_keyword['E_keyword'].values[1], query, sort, lastpage, period)
  df_default = pd.concat([df_default, df], axis = 0)


df_default.head()
df_default.to_csv('E부문_탄소_25_50.csv', header = True, index = False)

HMM
탄소
25
한국전력
탄소
26
LG
탄소
27
삼성생명
탄소
28
삼성전기
탄소
29
엔씨소프트
탄소
30
하나금융지주
탄소
31
삼성에스디에스
탄소
32
포스코케미칼
탄소
33
아모레퍼시픽
탄소
34
KT&G
탄소
35
대한항공
탄소
36
S-Oil
탄소
37
삼성화재
탄소
38
넷마블
탄소
39
두산중공업
탄소
40
하이브
탄소
41
고려아연
탄소
42
SK바이오팜
탄소
43
한온시스템
탄소
44
KT
탄소
45
롯데케미칼
탄소
46
한국조선해양
탄소
47
한화솔루션
탄소
48
우리금융지주
탄소
49


In [ ]:
df_default

,stockName,ESGKeyword,Title,Article
0,Test,Test,Test,Test
0,HMM,탄소,효성첨단소재·HMM·엘앤에프···25개株 '꿈의 10루타' 때렸다,[코로나 이후 18개월만에 대박]코스피 7곳·코스닥 18곳 텐배거 달성두산퓨얼셀·한...
1,HMM,탄소,"[기회의 K-바다] 국적선사 지원, 탄소중립…'해운 재건' 넘어 '해운 리더' 넘본다",올해 정부는 해운 분야 ‘리더 국가’의 목표를 세웠다. HMM의 세계 최대 컨테...
2,HMM,탄소,"잘 나가는 해운업계, 초대형·친환경 공격 행보로 승부수",[머니S리포트-진정한 G7을 향하여… 포스트 코로나 질주할 ‘넥스트-K’⑤] 10년...
3,HMM,탄소,<친환경이 경쟁력이다>석탄대신 수소로 쇳물 만들어… 2050년 탄소중립 실현,포스코는 수소 사업 밸류체인 구축 일환으로 광양제철소에 부생수소 생산설비를 가동하고...
...,...,...,...,...
380,우리금융지주,탄소,장 마감 후 기업공시[4월 13일],[서울경제] <유가 증권>▲지역난방공사(071320)=양승환 사외이사 의원 면직으로...
381,우리금융지주,탄소,"대세 된 ESG 경영, 여성 사외이사, 주목되는 미래 전략",[이코노미조선]세 가지 키워드로 살펴본 올해 슈퍼 주총3월 17일 경기도 수원컨벤션...
382,우리금융지주,탄소,친환경·미디어 플랫폼…상장사 미래먹거리 '주목',코스피 100대 기업 新사업 분석한화솔루션 그린수소 참여하고현대제철은 연료전지 사업...
383,우리금융지주,탄소,"[기업공시 4월 13일]셀트리온, 붙이는 치매 치료제 '도네페질 패취' 허가 신청 등",[서울경제] <유가 증권>▲지역난방공사(071320)=양승환 사외이사 의원 면직으로...


In [ ]:
df_default.to_csv('E부문_탄소_25_50.csv', header = True, index = False)